In [1]:
import numpy as np
import pandas as pd

In [2]:
from feature_selection import (construct_mutual_info_relevance_vector, 
                               construct_pearson_corr_relevance_vector,
                               construct_mutual_info_redundancy_matrix,
                               construct_pearson_corr_redundancy_matrix,
                               quadratic_programming_feature_selection,
                               greedy_mrmr_feature_selection,
                               generate_reduced_quadratic_program_with_qpfs)                              

In [3]:
feature_data = pd.read_csv("/Users/guomingwang/Desktop/BASF/dataset/df.csv", header="infer")
label_data = pd.read_csv("/Users/guomingwang/Desktop/BASF/dataset/Y.csv", header="infer")
# print(feature_data)
# print(label_data)

In [4]:
all_data = feature_data.join(label_data.set_index('id'), on='id')
# print(all_data)

In [5]:
index_to_label = list(sorted(set(all_data['class_vp'])))
label_to_index = {label:i for i, label in enumerate(index_to_label)}
label_to_index

{'Gas': 0, 'VOC with high vp': 1, 'VOC with low vp': 2}

In [6]:
class_id = all_data['class_vp'].apply(lambda x: label_to_index[x])
all_data.insert(len(all_data.columns), 'class_id', class_id)
# print(all_data)

In [7]:
X = all_data.values[:, :-4].astype(np.float32)
y = all_data.values[:, -1].astype(np.int32)
# print(X)
# print(y)

In [8]:
f_mi = construct_mutual_info_relevance_vector(X, y)
Q_mi = construct_mutual_info_redundancy_matrix(X)

In [9]:
f_pc = construct_pearson_corr_relevance_vector(X, y)
Q_pc = construct_pearson_corr_redundancy_matrix(X)

In [10]:
# num_of_chosen_features = 10

In [11]:
# qpfs_chosen_ones_mi, qpfs_feature_weights_mi = quadratic_programming_feature_selection(Q_all_mi, f_all_mi, num_of_chosen_features)
# print(qpfs_chosen_ones_mi)
# print(qpfs_feature_weights_mi)

In [12]:
# qpfs_chosen_ones_pc, qpfs_feature_weights_pc = quadratic_programming_feature_selection(Q_all_pc, f_all_pc, num_of_chosen_features)
# print(qpfs_chosen_ones_pc)
# print(qpfs_feature_weights_pc)

In [13]:
# mrmr_chosen_ones_mi, _, _, _ = greedy_mrmr_feature_selection(Q_all_mi, f_all_mi, num_of_chosen_features)
# print(mrmr_chosen_ones_mi)

In [14]:
# mrmr_chosen_ones_pc, _, _, _ = greedy_mrmr_feature_selection(Q_all_pc, f_all_pc, num_of_chosen_features)
# print(mrmr_chosen_ones_pc)

In [15]:
bar_q_mi = np.mean(Q_mi)
bar_f_mi = np.mean(f_mi)
alpha_mi = bar_q_mi / (bar_q_mi + bar_f_mi)
print(alpha_mi)

0.625011598376353


In [16]:
bar_q_pc = np.mean(Q_pc)
bar_f_pc = np.mean(f_pc)
alpha_pc = bar_q_pc / (bar_q_pc + bar_f_pc)
print(alpha_pc)

0.5475865536593721


In [17]:
num_frozen_features = 12
num_active_features = 8
frozen_vector_strategy = "hybrid"

In [46]:
# generate_reduced_quadratic_program_with_qpfs(Q_mi, f_mi, num_active_features, 
#                                              num_frozen_features=num_frozen_features, 
#                                              alpha = 1.5 * alpha_mi,                                             
#                                              frozen_vector_strategy=frozen_vector_strategy) 

In [45]:
# generate_reduced_quadratic_program_with_qpfs(Q_pc, f_pc, num_active_features, 
#                                              num_frozen_features=num_frozen_features, 
#                                              alpha = 1.5 * alpha_pc,                                             
#                                              frozen_vector_strategy=frozen_vector_strategy)

In [24]:
def get_bin(x, n=0):
    b = format(x, 'b').zfill(n)
    return np.array([int(a) for a in b])

In [32]:
def solve_qubo_by_brute_force(Q, v):
    d = len(v)
    opt_sol = None
    opt_val = None
    f = lambda x: 0.5 * np.dot(x, np.dot(Q, x)) + np.dot(v, x)
    for i in range(2**d):
        x = get_bin(i, n=d)
        if opt_val is None or f(x) < opt_val:
            opt_sol = x
            opt_val = f(x)
    return opt_sol, opt_val

In [33]:
# d = 10
# Q = np.random.uniform(0.0, 1.0, size=(d, d))
# f = np.random.uniform(0.0, 1.0, size=(d))
# solve_qubo_by_brute_force(Q, f)

In [34]:
def generate_reduced_quadratic_program_with_qpfs_for_alphas(Q, f, num_active_features, 
                                                             num_frozen_features=0, 
                                                             alphas=[],                                             
                                                             frozen_vector_strategy="QPFS"):
    
    for alpha in alphas:
        print("alpha = {}".format(alpha))
        Q1, f1, c1, df, af, ff = generate_reduced_quadratic_program_with_qpfs(Q, f, num_active_features, 
                                                 num_frozen_features=num_frozen_features,
                                                 alpha=alpha,
                                                 frozen_vector_strategy=frozen_vector_strategy)
        opt_sol, opt_val = solve_qubo_by_brute_force(Q1, f1)
        print("opt sol = {}".format(opt_sol)) 

In [53]:
num_active_features = 16
num_frozen_features = 10

Q = Q_mi
f = f_mi
alpha = alpha_mi

# Q = Q_pc
# f = f_pc
# alpha = alpha_pc

alphas = np.linspace(np.clip(alpha, 0.0, 1.0), np.clip(2.0*alpha_mi, 0.0, 1.0), 21)

# frozen_vector_strategy = "QUBO"
frozen_vector_strategy = "QPFS"
# frozen_vector_strategy = "hybrid"

generate_reduced_quadratic_program_with_qpfs_for_alphas(Q, f, num_active_features,
                                                        num_frozen_features=num_frozen_features, 
                                                        alphas=alphas,
                                                        frozen_vector_strategy=frozen_vector_strategy)

alpha = 0.625011598376353
     pcost       dcost       gap    pres   dres
 0: -2.0087e-01 -1.3379e+00  1e+02  1e+01  9e+00
 1: -1.4070e-01 -1.2682e+00  3e+00  1e-01  1e-01
 2: -1.2169e-01 -4.9698e-01  4e-01  2e-03  1e-03
 3: -1.7700e-01 -3.0108e-01  1e-01  6e-17  6e-16
 4: -1.8935e-01 -2.1983e-01  3e-02  2e-16  3e-16
 5: -1.9602e-01 -2.0561e-01  1e-02  2e-17  3e-16
 6: -1.9957e-01 -2.0154e-01  2e-03  2e-16  5e-16
 7: -2.0046e-01 -2.0075e-01  3e-04  6e-17  4e-16
 8: -2.0063e-01 -2.0064e-01  1e-05  6e-17  2e-16
 9: -2.0063e-01 -2.0063e-01  5e-07  2e-16  4e-16
10: -2.0063e-01 -2.0063e-01  3e-08  2e-16  4e-16
Optimal solution found.
opt sol = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
alpha = 0.6437610184575353
     pcost       dcost       gap    pres   dres
 0: -2.2217e-01 -1.3629e+00  1e+02  1e+01  8e+00
 1: -1.5370e-01 -1.2884e+00  3e+00  1e-01  1e-01
 2: -1.3310e-01 -5.2079e-01  4e-01  2e-03  2e-03
 3: -1.9485e-01 -3.2657e-01  1e-01  6e-17  7e-16
 4: -2.0875e-01 -2.4311e-01  3e-02  4e-16  7e-16

opt sol = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
alpha = 0.8687540594317236
     pcost       dcost       gap    pres   dres
 0: -7.6472e-01 -1.7947e+00  2e+02  1e+01  8e+00
 1: -3.2354e-01 -1.6217e+00  3e+00  1e-01  9e-02
 2: -2.9402e-01 -8.5358e-01  7e-01  2e-02  1e-02
 3: -5.2546e-01 -7.8957e-01  3e-01  2e-16  7e-16
 4: -6.4839e-01 -6.7726e-01  3e-02  1e-16  8e-16
 5: -6.7211e-01 -6.7328e-01  1e-03  1e-16  6e-16
 6: -6.7324e-01 -6.7325e-01  1e-05  2e-16  5e-16
 7: -6.7325e-01 -6.7325e-01  1e-07  7e-21  5e-16
Optimal solution found.
opt sol = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
alpha = 0.8875034795129059
     pcost       dcost       gap    pres   dres
 0: -8.6218e-01 -1.8441e+00  2e+02  1e+01  8e+00
 1: -3.2984e-01 -1.6630e+00  3e+00  1e-01  9e-02
 2: -3.0765e-01 -8.8591e-01  7e-01  2e-02  1e-02
 3: -5.6478e-01 -8.4608e-01  3e-01  4e-16  8e-16
 4: -7.0868e-01 -7.3454e-01  3e-02  1e-16  8e-16
 5: -7.3091e-01 -7.3140e-01  5e-04  9e-19  9e-16
 6: -7.3137e-01 -7.3138e-01  5e-06  2e-20  8e-16
 7: